# Multiclass Classification with SVM

Load a multiclass classification dataset and train a support vector machine (SVM) model. Classify the dataset using the trained model and print out the accuracy. Save the SVM model to svm.mlnet. Once completed, print 'task resolved' in the end.

## Task Description

Load a multiclass classification dataset and train a support vector machine (SVM) model. Classify the dataset using the trained model and print out the accuracy. Save the SVM model to svm.mlnet. Once completed, print 'task resolved' in the end.

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;

var mlContext = new MLContext(seed: 0);

// Generate random data
var dataPoints = GenerateRandomDataPoints(1000);
var data = mlContext.Data.LoadFromEnumerable(dataPoints);

// Map the label to key
var pipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
    .Append(mlContext.MulticlassClassification.Trainers.SdcaNonCalibrated())
    .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

// Train the model
var model = pipeline.Fit(data);

// Evaluate the model
var transformedData = model.Transform(data);
var metrics = mlContext.MulticlassClassification.Evaluate(transformedData);
Console.WriteLine($"Accuracy: {metrics.MacroAccuracy}");

// Save the model
mlContext.Model.Save(model, data.Schema, "svm.mlnet");

Console.WriteLine("task resolved");

public class ModelInput
{
    [ColumnName("Label"), LoadColumn(0)]
    public string Label { get; set; }

    [LoadColumn(1, 50)]
    [VectorType(50)]
    public float[] Features { get; set; }
}

private static IEnumerable<ModelInput> GenerateRandomDataPoints(int count, int seed = 0)
{
    var random = new Random(seed);
    float randomFloat() => (float)random.NextDouble();
    for (int i = 0; i < count; i++)
    {
        var label = randomFloat() > 0.5f ? "1" : "0";
        yield return new ModelInput
        {
            Label = label,
            Features = Enumerable.Repeat(label, 50).Select(x => x == "1" ? randomFloat() : randomFloat() + 0.1f).ToArray()
        };
    }
}